In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
#sigmoid fonk f(x)=1/(1+exp^-x)
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [4]:
def deriv_sigmoid(x):
    return(x*(1-x))

In [5]:
def relu(x):
    return np.maximum(0,x)

In [6]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [7]:
def initialize(x, neuron_num):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num))-1
    w2=2*np.random.random((neuron_num, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    return w1, w2, bias1, bias2

In [8]:
def forward_prop(w1, w2, bias1, bias2, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    return z1, out1, z2, out2

In [9]:
def back_prop(w1, w2, bias1, bias2, x, y, z1, out1, z2, out2, lr, j):
    x=x.reshape(13,1)
    err_2=2*(np.subtract( out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(1,1)
    dc_dw2=lr*np.dot(out1,dummy_2)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(y, out1.sum(axis=0)))
    dummy_1=np.array(deriv_relu(z1)).reshape(7,1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, bias1, bias2

In [10]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [11]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [12]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [13]:
#ds.head()

In [14]:
#ds.isnull().sum()

In [15]:
#ds.describe(include='all')

In [16]:
target=ds.loc[:,'MEDV'].copy()

In [17]:
data=ds.drop('MEDV', axis=1).copy()

In [18]:
w1, w2, bias1, bias2=initialize(data,7)

In [19]:
scaler=StandardScaler()
data_normalized=scaler.fit_transform(data)
target=target.to_numpy()

In [20]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [21]:
epochs=5

In [22]:
lr=0.1

In [23]:
output1=list()
output2=list()
z_1=list()
z_2=list()
loss=0
loss_test=0

In [24]:
for e in range(epochs):
    output1.clear()
    output2.clear()
    z_1.clear()
    z_2.clear()
    errors=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2= forward_prop(w1, w2, bias1, bias2, x_train[i])
        output1.append(out1)
        output2.append(out2)
       # predicted[i]=output[i].sum(axis=0)
        w1, w2, bias1, bias2 = back_prop(w1, w2, bias1, bias2, x_train[i], y_train[i], z1, output1[i], z2, output2[i], lr, i+1)

In [25]:
for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
    loss+=(y_train[i]-(output2[i].sum(axis=0)))

In [26]:
print(loss/(len(y_train)))

[-5.27630744e-16]


In [27]:
out_test=list()
for i in range(len(y_test)): 
    z_test1, out_test1, z_test2, out_test2=forward_prop(w1, w2, bias1, bias2, x_test[i])
    out_test.append(out_test2)
    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
print(loss_test/(len(y_test)))

[6.91328919]


In [28]:
for i in range(len(y_test)):
    print(y_test[i], out_test[i])

8.5 [[9.11220101]]
5.0 [[9.11220101]]
11.9 [[9.11220101]]
27.9 [[9.11220101]]
17.2 [[9.11220101]]
27.5 [[9.11220101]]
15.0 [[9.11220101]]
17.2 [[9.11220101]]
17.9 [[9.11220101]]
16.3 [[9.11220101]]
7.0 [[9.11220101]]
7.2 [[9.11220101]]
7.5 [[9.11220101]]
10.4 [[9.11220101]]
8.8 [[9.11220101]]
8.4 [[9.11220101]]
16.7 [[9.11220101]]
14.2 [[9.11220101]]
20.8 [[9.11220101]]
13.4 [[9.11220101]]
11.7 [[9.11220101]]
8.3 [[9.11220101]]
10.2 [[9.11220101]]
10.9 [[9.11220101]]
11.0 [[9.11220101]]
9.5 [[9.11220101]]
14.5 [[9.11220101]]
14.1 [[9.11220101]]
16.1 [[9.11220101]]
14.3 [[9.11220101]]
11.7 [[9.11220101]]
13.4 [[9.11220101]]
9.6 [[9.11220101]]
8.7 [[9.11220101]]
8.4 [[9.11220101]]
12.8 [[9.11220101]]
10.5 [[9.11220101]]
17.1 [[9.11220101]]
18.4 [[9.11220101]]
15.4 [[9.11220101]]
10.8 [[9.11220101]]
11.8 [[9.11220101]]
14.9 [[9.11220101]]
12.6 [[9.11220101]]
14.1 [[9.11220101]]
13.0 [[9.11220101]]
13.4 [[9.11220101]]
15.2 [[9.11220101]]
16.1 [[9.11220101]]
17.8 [[9.11220101]]
14.9 [[9.112

In [29]:
for i in range(len(y_train)):
    print(y_train[i], output2[i])

24.0 [[9.11220101]]
21.6 [[12.08976081]]
34.7 [[13.99180865]]
33.4 [[18.13344692]]
36.2 [[21.18675753]]
28.7 [[24.18940603]]
22.9 [[25.09152482]]
27.1 [[24.65321986]]
16.5 [[25.14257589]]
18.9 [[23.41406071]]
15.0 [[22.51124857]]
18.9 [[21.00899885]]
21.7 [[20.58719908]]
20.4 [[20.80975927]]
18.2 [[20.72780741]]
19.9 [[20.22224593]]
23.1 [[20.15779674]]
17.5 [[20.7462374]]
20.2 [[20.09698992]]
18.2 [[20.11759193]]
13.6 [[19.73407355]]
19.6 [[18.50725884]]
15.2 [[18.72580707]]
14.5 [[18.02064566]]
15.6 [[17.31651652]]
13.9 [[16.97321322]]
16.6 [[16.35857058]]
14.8 [[16.40685646]]
18.4 [[16.08548517]]
21.0 [[16.54838813]]
12.7 [[17.43871051]]
14.5 [[16.49096841]]
13.2 [[16.09277473]]
13.1 [[15.51421978]]
13.5 [[15.03137582]]
18.9 [[14.72510066]]
20.0 [[15.56008053]]
21.0 [[16.44806442]]
24.7 [[17.35845154]]
30.8 [[18.82676123]]
34.9 [[21.22140898]]
26.6 [[23.95712719]]
25.3 [[24.48570175]]
24.7 [[24.6485614]]
21.2 [[24.65884912]]
19.3 [[23.9670793]]
20.0 [[23.03366344]]
16.6 [[22.4269307